# League of Legends Data Analysis
### My self-taught python journey

I've always been curious to drill deeper into the details of League of Legends performance than available through the standard platforms. As I teach myself Python, I want to dive deeper into what makes a Solo-queue winner: how do top performers compare the the rest of us in terms of damage share, consistency, and macro play?

## Step 0: Import Necessary Modules

In order to use any of this code, you'll need to get a key from https://developer.riotgames.com/. 

In [1]:
# @hidden_cell
key = 'redacted per Riot Guidelines'

In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests
import json
import time
import matplotlib as plt
from datetime import datetime

In [3]:
url_Core = 'https://na1.api.riotgames.com'
url_Summ_By_Name = '/lol/summoner/v4/summoners/by-name/'
url_Match_History = '/lol/match/v4/matchlists/by-account/'
url_Match_Stats = '/lol/match/v4/matches/'
url_Match_Timeline = '/lol/match/v4/timelines/by-match/'

In [4]:
champion_dict = 'http://ddragon.leagueoflegends.com/cdn/10.1.1/data/en_US/champion.json'

In [5]:
lookup_name = 'jigglemyjag'

In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Step 1: Import Necessary Data

In [7]:
def summRequest(name):
    """Pulls identifying summoner information to be used in later methods.
    In: name
    Out: JSON dictionary"""
    url = f'{url_Core}{url_Summ_By_Name}{name}?api_key={key}'
    response = requests.get(url).json()
    #dataframe = json_normalize(response)
    return response

#### Explore the Match History Options 

In [8]:
json_normalize(summRequest(lookup_name))

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,SBBIrs6Aue0Nxu-uCkGafeHvzh_Pf25okIMwQAro-ddkB4w,D6vSjql1vG4RsEXLbzFueyKjizX8tMm6HWfUFFpKand1kg,O5cnL8OBx44QpOC01b-5K36rxKW0jZkSJdK5GJBr6UBATv...,JiggleMyJag,4403,1578632455000,84


Note that we <b>can improve our summRequest function by addint startIndex and endIndex functionality

In [9]:
def getMatchHistory(name, champion=None, queue=None, lane=None):
    paramkey = {'champion': champion, 'queue': queue, 'api_key': key}
    acct_ID = summRequest(name)['accountId']
    url = f'{url_Core}{url_Match_History}{acct_ID}'
    response = requests.get(url, params=paramkey).json()
#        print("No games meet these search results")
    response = pd.DataFrame.from_records(response)
    response = response.drop(columns = 'matches').assign(**pd.DataFrame.from_records(response['matches'].dropna().tolist()))
    response = response[response.lane==lane.upper()] if lane != None else response
    
    response.drop(columns=['endIndex', 'startIndex', 'totalGames', 'role', 'lane'], inplace=True)
    
    if len(response) == 0:
        print("No games meet these search results")
        

    return response

In regard to dropping the role and lane columns above; this information is provided later, and has been checked that it provides the same information. The only difference is in how the two sources handle the middle lane; the above source refers to it as "mid" and the other as "middle"

In [10]:
x = getMatchHistory('JiggleMyJag')
display(x.head(1))
del(x)

,platformId,gameId,champion,queue,season,timestamp
0,NA1,3259063988,15,450,13,1578631190237


In [11]:
def matchStats(matchID):
    """Accesses Riot's API for specific match stats, which requires a match Id
    In: gameId int
    Out: JSON dictionary"""
    url = f'{url_Core}{url_Match_Stats}{matchID}?api_key={key}'
    response = requests.get(url).json()
    return response

In [12]:
%timeit getMatchHistory('JiggleMyJag', queue=400)

466 ms ± 27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
match_hist = getMatchHistory('JiggleMyJag', queue= 400)
match_hist.head(5)

,platformId,gameId,champion,queue,season,timestamp
0,NA1,3256328496,81,400,13,1578369973114
1,NA1,3256322204,29,400,13,1578367569880
2,NA1,3255526755,236,400,13,1578276212286
3,NA1,3253600534,81,400,13,1578105695357
4,NA1,3253573330,81,400,13,1578103787262


In [14]:
def matchHistoryStats(name, champion=None, queue=None, lane=None):
    """returns a dictionary unpacking a game's match stats 
    In: list of gameId integers
    Out: dataframe match history with following headers"""
    hist = getMatchHistory(name, champion, queue, lane)
    lst = [matchStats(game) for game in hist.gameId.tolist()[0:98]]
    stats_df = pd.DataFrame.from_records(lst)
    stats_df.rename(columns={'seasonId':'season', 'queueId':'queue'}, inplace=True)
    out_df = hist.merge(stats_df, on= ['gameId', 'platformId', 'queue', 'season'])#.reset_index()#.drop(columns='level_0')
    return(out_df)

In [15]:
time.sleep(123)
start = datetime.now()
match_hist_stats = matchHistoryStats('JiggleMyJag', queue = 420)
now = datetime.now() - start
print(f'Creating the initial match history dataframe took {now.seconds}.{now.microseconds} seconds')

Creating the initial match history dataframe took 11.872021 seconds


# Unpacking Dictionaries within a Dataframe

#### Step 1: Data Exploration
Let's first explore the dataframe so you know what I'm talking about. While the first columns may appear standard, the final three contain nested lists of dictionaries - how annoying! Obviously we can make separate dataframes with these contents, but we would then lose our index variable (gameId)

In [16]:
match_hist_stats.head(2)

,platformId,gameId,champion,queue,season,timestamp,gameCreation,gameDuration,mapId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,NA1,3204748560,145,420,13,1573610093349,1573610093349,1811,11,9.22.296.5720,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,NA1,3190462948,77,420,13,1572408447992,1572408447992,1871,11,9.21.294.813,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


#### Step 2: Unpacking

In [17]:
def masterFrame(df):
    """Unpacks nested dictionaries within the participants, participantIdentities, and teams columns 
    using a multiIndex w/GameId. Deletes above columns after unpacking, and also removes other misc
    unused columns as well."""
    frame = df
    cols = ['participants', 'participantIdentities', 'teams']
    indexes = [['gameId'], ['gameId', 'participantId'], ['gameId', 'teamId']]
    for col, index in zip(cols, indexes):
        #idx = df.set_index('gameId')[col].apply(pd.Series).stack().index
        idx = df.set_index(['gameId'])[col].apply(pd.Series).stack().index #[match_hist.index, match_hist.gameId], index= x
        temp = pd.DataFrame(df[col].apply(pd.Series).stack().values.tolist(), index = idx).reset_index().drop('level_1', 1)
        frame = frame.merge(temp, on = index, how = 'left')
        
    frame.drop(columns=cols, inplace=True)    
    for i in ['stats', 'timeline', 'player']:
        frame = frame.drop(i, 1).assign(**pd.DataFrame.from_records(frame[i].dropna().tolist()))
    
    frame.drop(columns = ['combatPlayerScore', 'objectivePlayerScore','totalPlayerScore', 'totalScoreRank', 
    'playerScore0', 'playerScore1', 'playerScore2', 'playerScore3','playerScore4', 
    'playerScore5', 'playerScore6', 'playerScore7', 'playerScore8', 'playerScore9'], inplace=True)
    
    frame.set_index(['gameId', 'summonerName', 'participantId'], inplace=True)
    frame.reset_index(inplace=True)
               
    return frame

In [18]:
masterFrame(match_hist_stats)[:12:4]

,gameId,summonerName,participantId,platformId,champion,queue,season,timestamp,gameCreation,gameDuration,mapId,gameVersion,gameMode,gameType,teamId,championId,spell1Id,spell2Id,highestAchievedSeasonTier,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,totalMinionsKilled,neutralMinionsKilled,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,wardsPlaced,wardsKilled,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,firstInhibitorKill,firstInhibitorAssist,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2,creepsPerMinDeltas,xpPerMinDeltas,goldPerMinDeltas,csDiffPerMinDeltas,xpDiffPerMinDeltas,damageTakenPerMinDeltas,damageTakenDiffPerMinDeltas,role,lane,accountId,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon
0,3204748560,Phae,1,NA1,145,420,13,1573610093349,1573610093349,1811,11,9.22.296.5720,CLASSIC,MATCHED_GAME,100,412,14,4,NaN,True,False,True,True,False,True,True,10,0,1,4,0,1,0,"[{'championId': 25, 'pickTurn': 1}, {'champion...",3117,3069,2065,3107,3800,0,3364,6,5,19,2,2,2,867,1,0,0,0,0,22465,15652,5144,1668,0,9718,7642,903,1172,2487,8,14276,4341,1834,64,39,17624,8492,8717,414,11561,9800,1,9,4,0,0,151,14,4,0,26,6,False,False,False,True,False,True,8439,919,0,0,8463,841,0,0,8473,419,0,0,8242,11,0,0,8347,0,0,0,8306,5,0,0,8400,8300,5008.0,5008.0,5003.0,"{'10-20': 0.5, '0-10': 0, '20-30': 0.4}","{'10-20': 338, '0-10': 299.5, '20-30': 595.5}","{'10-20': 397, '0-10': 234.9, '20-30': 459.3}","{'10-20': -3.4000000000000004, '0-10': 0.29999...","{'10-20': -117.25000000000003, '0-10': 59.6999...","{'10-20': 720.7, '0-10': 144.4, '20-30': 856.9}","{'10-20': -51.19999999999999, '0-10': -127, '2...",DUO_SUPPORT,BOTTOM,LJl9nOBBBZugO4G8TdmR5LeGboDQjuZ0t6_UJvw5YlthMMk,SnHVvi_CUsFtO5PaIBHK-nENbFiNkZuxuNYIY6N4EHbTUNo,NA1,LJl9nOBBBZugO4G8TdmR5LeGboDQjuZ0t6_UJvw5YlthMMk,/v1/stats/player_history/NA1/234553081,23
4,3204748560,Devilish Bronze,5,NA1,145,420,13,1573610093349,1573610093349,1811,11,9.22.296.5720,CLASSIC,MATCHED_GAME,100,246,4,11,NaN,True,False,True,True,False,True,True,10,0,1,4,0,1,0,"[{'championId': 25, 'pickTurn': 1}, {'champion...",1412,2031,3111,3147,3134,3133,3340,6,7,5,2,1,1,653,0,0,0,0,0,122597,17554,98266,6775,0,10282,800,9254,227,11666,1,13578,11532,2181,26,16,28453,7801,20461,190,10602,9050,3,9,119,72,28,187,15,1,0,10,3,False,False,False,False,False,True,8112,476,0,0,8143,189,0,0,8138,18,0,0,8135,3659,5,0,9104,14,20,0,9111,496,220,0,8100,8000,5005.0,5003.0,5002.0,"{'10-20': 0.30000000000000004, '0-10': 0.1, '2...","{'10-20': 426.9, '0-10': 274.5, '20-30': 647.2}","{'10-20': 379.4, '0-10': 248.1, '20-30': 343.4}","{'10-20': -0.19999999999999998, '0-10': 0.1, '...","{'10-20': 25.900000000000006, '0-10': -48.8999...","{'10-20': 1070.7, '0-10': 729.3, '20-30': 997.4}","{'10-20': 218.79999999999995, '0-10': 251.2000...",NONE,JUNGLE,4POgGz7vnwCw8rsON5-tohsSfSsoQlJ_6e48Bio6rooMzUQ,ng9L0N0wXm7VSDr3Wr6GtwvA-G0cwJpniaui81LpX6pEcKg,NA1,

In [19]:
%timeit masterFrame(match_hist_stats)

366 ms ± 38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Let's make sure the output looks right: Yup, it does. <p> 
    
After scanning the dataframe, there are some columns that look like they're not used: <em> combatPlayerScore	objectivePlayerScore	totalPlayerScore	totalScoreRank	playerScore0	playerScore1	playerScore2	playerScore3	playerScore4	playerScore5	playerScore6	playerScore7	playerScore8	playerScore9. </em> <p>
    
I've gone ahead and added a line in the method to remove the aforementioned columns.

##### A question for those more familiar with Python: why do I need to convert values.tolist() to make the above function work? It seems to construct single dataframes just fine in isolation, but once I put it the combined function it gives me a key error...

I actually do not know 

# Section 3: Analysis
Now that we've got our data wrangled into a master frame, we can get to our analysis. </p>

Let's identifiy some of our goals:
- How much damage does contribute summoner contribute in victories vs defeats?
- Kill and Death locations in wins vs losses

#### Analysis 3.1 Damage Share

In [20]:
def winLossDamageShare(name, aggregated_df, lane=None):
    """Displays the average total damage done by the summoner, grouped by wins and losses.
        Win: False indicates stats in a loss.
        Win: True indicates stats in a win"""
    
    summ_filter = aggregated_df['summonerName'].str.lower() == name.lower()             #str.lower() for dataframes
    lane_filter = aggregated_df['lane'] if lane is None else aggregated_df['lane'] == lane.upper()
    
    lst = [(row.gameId, row.teamId) for key, row in aggregated_df[
        (summ_filter) & (lane_filter) & (aggregated_df.lane != 'NONE') & (aggregated_df.role != 'DUO')].iterrows()]
    df_lst = [row for key, row in aggregated_df.iterrows() if ((row.gameId, row.teamId) in lst)]
    dfallies = pd.DataFrame(df_lst)

    #print messages
    print(f"{name}'s Damage Analsysis:")
    """print(aggregated_df[summ_filter & lane_filter].groupby('win').agg({'totalDamageDealtToChampions': [np.mean, np.size]}))
    print('\n\n %Team Damage Share in Wins and Losses')
    print( aggregated_df[summ_filter & lane_filter].groupby('win').sum().totalDamageDealtToChampions /
    dfallies.groupby('win').sum().totalDamageDealtToChampions)
"""
    #print(aggregated_df[summ_filter].groupby('win').agg({'totalDamageDealtToChampions': [np.mean, np.size]}))
    print('\n\n %Team Damage Share in Wins and Losses by role')
    print(
        aggregated_df[(summ_filter) & (aggregated_df.lane != 'NONE') & (aggregated_df.role != 'DUO')].groupby(
        ['lane', 'role', 'win']).sum().totalDamageDealtToChampions / dfallies.groupby(['win']).sum().totalDamageDealtToChampions
        , aggregated_df[(summ_filter) & (aggregated_df.lane != 'NONE') & (aggregated_df.role != 'DUO')].groupby(
        ['lane', 'role', 'win']).size()
    )
    
    """aggregated_df[(summ_filter) & (aggregated_df.lane != 'NONE') & (aggregated_df.role != 'DUO')].groupby(
        ['lane', 'role', 'win']).agg({'totalDamageDealtToChampions': [np.sum, np.size]})"""
    
    return

The below cell displays the average total damage done by the summoner (using the specified queue and champion filters from earlier) by wins and losses. 

<p> <b> Win: False</b> indicates stats in a loss. </p>
<p> <b> Win: True</b> indicates stats in a win </p>

In [21]:
master = masterFrame(match_hist_stats)

In [22]:
print(type(master.timestamp.tolist()[1]))

<class 'int'>


In [23]:
master[(master.lane == 'TOP') & (master.summonerName == 'JiggleMyJag')]

,gameId,summonerName,participantId,platformId,champion,queue,season,timestamp,gameCreation,gameDuration,mapId,gameVersion,gameMode,gameType,teamId,championId,spell1Id,spell2Id,highestAchievedSeasonTier,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,totalMinionsKilled,neutralMinionsKilled,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,wardsPlaced,wardsKilled,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,firstInhibitorKill,firstInhibitorAssist,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2,creepsPerMinDeltas,xpPerMinDeltas,goldPerMinDeltas,csDiffPerMinDeltas,xpDiffPerMinDeltas,damageTakenPerMinDeltas,damageTakenDiffPerMinDeltas,role,lane,accountId,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon


^^ The above code indicates that I have not played any games in the top lane; we should see this reflected in the role/lane breakdown below. 

In [24]:
display(winLossDamageShare('jigglemyjag', aggregated_df=master))

jigglemyjag's Damage Analsysis:


 %Team Damage Share in Wins and Losses by role
lane    role       win  
BOTTOM  DUO_CARRY  False    0.186175
                   True     0.187858
        SOLO       False    0.006552
JUNGLE  NONE       True     0.004031
Name: totalDamageDealtToChampions, dtype: float64 lane    role       win  
BOTTOM  DUO_CARRY  False    24
                   True     20
        SOLO       False     1
JUNGLE  NONE       True      1
dtype: int64


None

# That's it for now - check in later for more analysis and data vizualization :) 

time.sleep(121)

friends = ['kyle']#, 'tarren', 'christian', 'shaggy']#, 'corrine', 'andrew']
summ_names = ['jigglemyjag']#, 'dragonfires123', 'queeermo', 'christianlovemen']#, 'Uitravioiet', 'shin getter robo']
pros = ['Doublelift', 'WildTurtle ', 'C9 Sneaky']
#friends.extend(pros)
#summ_names.extend(pros)

for friend_name, summ_name in zip(friends, summ_names):
    print(datetime.now())
    print(f"Compiling {friend_name.capitalize()}'s ranked stats data.")
    if friend_name in pros: 
        frame_ranked = masterFrame(matchHistoryStats(name=summ_name, queue=420, lane = 'bottom'))
    else: 
        frame_ranked = masterFrame(matchHistoryStats(name=summ_name, queue=420))
    frame_ranked.to_pickle(f'{friend_name} ranked solo queue data.pkl')
    print(f"{friend_name.capitalize()}'s dataframe finished at {datetime.now()}.")
    #time.sleep(125)

In [25]:
friends = ['kyle', 'tarren', 'christian', 'shaggy', 'corrine', 'andrew']
summ_names = ['jigglemyjag', 'dragonfires123', 'queeermo', 'christianlovemen', 'Uitravioiet', 'shin getter robo']
kyle = pd.read_pickle('kyle ranked solo queue data.pkl')
tarren = pd.read_pickle('tarren ranked solo queue data.pkl')
josh = pd.read_pickle('Ninjawolf07 ranked solo queue data.pkl')
#chris = pd.read_pickle('chris ranked solo queue data.pkl')
#shag = pd.read_pickle('shag ranked solo queue data.pkl')
#cor = pd.read_pickle('cor ranked solo queue data.pkl')
andrew = pd.read_pickle('andrew ranked solo queue data.pkl')
doublelift = pd.read_pickle('Doublelift ranked solo queue data.pkl')
#wildturtle = pd.read_pickle('WildTurtle ranked solo queue data.pkl')
sneaky = pd.read_pickle('C9 Sneaky ranked solo queue data.pkl')

In [37]:
summ_names = ['Jigglemyjag', 'Shin getter robo', 'Doublelift']
friend_dfs = [kyle, andrew, doublelift]

for summ, df in zip(summ_names, friend_dfs):
    winLossDamageShare(name=summ, aggregated_df=df)


Jigglemyjag's Damage Analsysis:


 %Team Damage Share in Wins and Losses by role
lane    role       win  
BOTTOM  DUO_CARRY  False    0.186175
                   True     0.187858
        SOLO       False    0.006552
JUNGLE  NONE       True     0.004031
Name: totalDamageDealtToChampions, dtype: float64 lane    role       win  
BOTTOM  DUO_CARRY  False    24
                   True     20
        SOLO       False     1
JUNGLE  NONE       True      1
dtype: int64
Shin getter robo's Damage Analsysis:


 %Team Damage Share in Wins and Losses by role
lane    role         win  
JUNGLE  NONE         False    0.153922
                     True     0.140448
TOP     DUO_SUPPORT  True     0.004937
Name: totalDamageDealtToChampions, dtype: float64 lane    role         win  
JUNGLE  NONE         False    47
                     True     44
TOP     DUO_SUPPORT  True      1
dtype: int64
Doublelift's Damage Analsysis:


 %Team Damage Share in Wins and Losses by role
lane    role         win  
BOTTOM  

In [38]:
def appendFrames(lst):
    df = pd.DataFrame()
    for frame in lst[1:]:
        df.append(frame[['summonerName', 'timestamp', 'gameVersion', 'championId', 'totalDamageDealtToChampions', 'creepsPerMinDeltas', 'csDiffPerMinDeltas']])

    return df
        

In [40]:
friend_dfs = [kyle, andrew, doublelift]

x = appendFrames(friend_dfs)

In [41]:
x.head(5)

""
